### Meta-Llama-3-8B-Instruct

In [1]:
import transformers
import pandas as pd
import os, sys
import time
import logging
import json
from tqdm import tqdm
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import torch
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

prompt_template = "Identify keywords from the summary and description of the bug report that can be used to detect duplicates.\n\nOutput format:\nSummary: [Selected Keywords]\nDescription: [Selected Keywords]\n\nSummary: {}\nDescription: {}\n\n"
project = 'spark'

df = pd.read_csv('../data/raw/test_{}.csv'.format(project))
flag_content_df = pd.read_csv(f'../data/ablation/test_{project}_flag_content.csv')

llama3_folder = '../data/keywords/{}/llama3/run_{}'

for run in range(1, 6):
    if not os.path.exists(llama3_folder.format(project, run)):
        os.makedirs(llama3_folder.format(project, run))

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        bug_id = row['bug_id']
        
        if flag_content_df[flag_content_df['bug_id'] == bug_id]['run_flag'].values[0] == 0:
            continue
        
        if os.path.exists(os.path.join(llama3_folder.format(project, run), f'{bug_id}.txt')):
            continue
                
        messages = [
            # {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
            {
                "role": "user", 
                "content": prompt_template.format(row['short_desc'], row['description'])
            },
        ]

        prompt = pipeline.tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
        )

        terminators = [
            pipeline.tokenizer.eos_token_id,
            pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]
        try:
            outputs = pipeline(
                prompt,
                max_new_tokens=2048,
                # max_length=2048,
                eos_token_id=terminators,
                do_sample=False,
                top_p=1,
            )
        except Exception as e:
                print(e)
                messages = [
                    {
                        "role": "user", 
                        "content": prompt_template.format(row['short_desc'], row['description'][:2000])
                    },
                ]

                prompt = pipeline.tokenizer.apply_chat_template(
                        messages, 
                        tokenize=False, 
                        add_generation_prompt=True
                )
                
                outputs = pipeline(
                    prompt,
                    max_new_tokens=2048,
                    # max_length=2048,
                    eos_token_id=terminators,
                    do_sample=False,
                    top_p=1,
                )
                
        with open(os.path.join(llama3_folder.format(project, run), f'{bug_id}.txt'), 'w') as f:
            f.write(prompt_template.format(row['short_desc'], row['description']))
            f.write('\n\n>>>>>> Response:\n\n')
            f.write(outputs[0]["generated_text"][len(prompt):])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  0%|          | 0/2841 [00:00<?, ?it/s]/home/tingzhang/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:484: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
 10%|▉         | 279/2841 [01:26<27:40,  1.54it/s]/home/tingzhang/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
 51%|█████     | 1444/2841 [21:27<29:34,  1.27s/it]Setting `pad_token_id` to `eos_token_id`:128009

The size of tensor a (16384) must match the size of tensor b (16882) at non-singleton dimension 3


 55%|█████▌    | 1572/2841 [47:44<3:51:52, 10.96s/it]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


CUDA out of memory. Tried to allocate 9.90 GiB (GPU 0; 47.51 GiB total capacity; 30.98 GiB already allocated; 5.49 GiB free; 41.51 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


...

 73%|███████▎  | 2068/2841 [7:09:07<2:49:13, 13.13s/it]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


CUDA out of memory. Tried to allocate 13.19 GiB (GPU 0; 47.51 GiB total capacity; 37.52 GiB already allocated; 2.89 GiB free; 44.11 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


  3%|▎         | 72/2841 [32:25<26:19:41, 34.23s/it]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


CUDA out of memory. Tried to allocate 139.25 GiB (GPU 0; 47.51 GiB total capacity; 25.14 GiB already allocated; 21.16 GiB free; 25.84 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


  5%|▌         | 154/2841 [3:33:51<61:32:26, 82.45s/it]  Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


### Phi-3-mini-128k-instruct

In [3]:
import pandas as pd
from transformers import AutoTokenizer
from tqdm import tqdm

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import torch
torch.random.manual_seed(42)
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
prompt_template = "Identify keywords from the summary and description of the bug report that can be used to detect duplicates.\n\nOutput format:\nSummary: [Selected Keywords]\nDescription: [Selected Keywords]\n\nSummary: {}\nDescription: {}\n\n"
project = 'spark'

df = pd.read_csv('../data/raw/test_{}.csv'.format(project))
flag_content_df = pd.read_csv(f'../data/ablation/test_{project}_flag_content.csv')

phi_folder = '../data/keywords/{}/phi/run_{}'

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")


for run in range(1, 6):
    if not os.path.exists(phi_folder.format(project, run)):
        os.makedirs(phi_folder.format(project, run))

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        bug_id = row['bug_id']
        
        if flag_content_df[flag_content_df['bug_id'] == bug_id]['run_flag'].values[0] == 0:
            continue
        
        if os.path.exists(os.path.join(phi_folder.format(project, run), f'{bug_id}.txt')):
            continue
                
        messages = [
            {
                "role": "user", 
                "content": prompt_template.format(row['short_desc'], row['description'])
            },
        ]

        try:
            pipe = pipeline(
                "text-generation",
                model=model,
                tokenizer=tokenizer,
            )
            
            generation_args = {
                "max_new_tokens": 2048,
                "return_full_text": False,
                "temperature": 0.0,
                "do_sample": False,
                "top_p": 1,
            }

            output = pipe(messages, **generation_args)
            # print(output[0]['generated_text'])
        
        except Exception as e:
                print(e)
                messages = [
                    {
                        "role": "user", 
                        "content": prompt_template.format(row['short_desc'], row['description'][:2000])
                    },
                ]

                pipe = pipeline(
                    "text-generation",
                    model=model,
                    tokenizer=tokenizer,
                )
                
                generation_args = {
                    "max_new_tokens": 2048,
                    "return_full_text": False,
                    "temperature": 0.0,
                    "do_sample": False,
                    "top_p": 1,
                }

                output = pipe(messages, **generation_args)
                # print(output[0]['generated_text'])
                
        with open(os.path.join(phi_folder.format(project, run), f'{bug_id}.txt'), 'w') as f:
            f.write(prompt_template.format(row['short_desc'], row['description']))
            f.write('\n\n>>>>>> Response:\n\n')
            f.write(output[0]['generated_text'])

/mnt/hdd1/tingzhang/anaconda3/envs/cupid/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.21s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  0%|          | 0/2841 [00:00<?, ?it/s]/mnt/hdd1/tingzhang/anaconda3/envs/cupid/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
 12%|█▏        | 343/2841 [08:05<4:33:51,  6.58s/it] 

CUDA out of memory. Tried to allocate 11.36 GiB. GPU 


 12%|█▏        | 345/2841 [08:15<4:13:31,  6.09s/it]

CUDA out of memory. Tried to allocate 11.36 GiB. GPU 


 12%|█▏        | 346/2841 [08:22<4:19:57,  6.25s/it]

...

CUDA out of memory. Tried to allocate 20.08 GiB. GPU 


 73%|███████▎  | 2068/2841 [1:56:02<07:37,  1.69it/s]  

CUDA out of memory. Tried to allocate 22.18 GiB. GPU 


 79%|███████▉  | 2255/2841 [2:06:56<32:59,  3.38s/it]  


KeyboardInterrupt: 